In [3]:
from sklearn.datasets import fetch_california_housing

In [4]:
data = fetch_california_housing()

In [5]:
x = data.data
y = data.target

x.shape, y.shape

((20640, 8), (20640,))

In [6]:
print(data.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [7]:
import pandas as pd

In [8]:
pd.DataFrame(x, columns=data.feature_names).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [9]:
pd.DataFrame(y, columns=data.target_names).head() # 집값의 중간값

,MedHouseVal
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422


1. Sequential API로 작성
2. 전처리(전처리 필요한지 확인)
3. 모델링(input-output구조, 히든레이어 최소 2개 이상 추가)
4. 학습(validation set 만들기, 분할 정도는 자율, 7번 성능 개선 없으면 멈추기, 학습회수는 500번)

In [20]:
pd.DataFrame(x, columns=data.feature_names).describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [22]:
pd.DataFrame(x, columns=data.feature_names).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [10]:
# 1. 전처리
# - train/test 분리
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=2022)

# - scailing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [21]:
train_x

array([[0.14141184, 0.49019608, 0.04722931, ..., 0.00365758, 0.15138593,
        0.59661355],
       [0.54435111, 0.15686275, 0.10730753, ..., 0.00355597, 0.13859275,
        0.65338645],
       [0.21807285, 0.43137255, 0.05186706, ..., 0.00229734, 0.50426439,
        0.24003984],
       ...,
       [0.17836306, 0.66666667, 0.06317818, ..., 0.00254374, 0.16098081,
        0.59561753],
       [0.14799106, 0.49019608, 0.0749678 , ..., 0.00525905, 0.56289979,
        0.30677291],
       [0.15267376, 0.31372549, 0.07861838, ..., 0.00361854, 0.70469083,
        0.23505976]])

In [23]:
# 2. 모델링
from tensorflow import keras

# 1) 세션 클리어 
keras.backend.clear_session()

# 2) 모델 생성
model = keras.models.Sequential()

# 3) 레이어 추가
model.add(keras.layers.Input(8, ))
model.add(keras.layers.Dense(1))

# 4) 컴파일
model.compile(loss='mse', optimizer='adam')

# 5) 구조 확인
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 3. 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   restore_best_weights=True)

model.fit(train_x, train_y, validation_split=0.15, callbacks=[es], epochs=500, verbose=1)

Epoch 1/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5732 - val_loss: 0.5781
Epoch 2/500
384/384 [==============================] - 1s 3ms/step - loss: 0.5720 - val_loss: 0.5774
Epoch 3/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5709 - val_loss: 0.5760
Epoch 4/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5697 - val_loss: 0.5751
Epoch 5/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5690 - val_loss: 0.5739
Epoch 6/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5678 - val_loss: 0.5728
Epoch 7/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5668 - val_loss: 0.5718
Epoch 8/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5660 - val_loss: 0.5710
Epoch 9/500
384/384 [==============================] - 1s 3ms/step - loss: 0.5651 - val_loss: 0.5700
Epoch 10/500
384/384 [==============================] - 1s 2ms/step - loss: 0.5642 - val_lo

In [28]:
# 2. 모델링
from tensorflow import keras

# 1) 세션 클리어 
keras.backend.clear_session()

# 2) 모델 생성
model = keras.models.Sequential()

# 3) 레이어 추가
model.add(keras.layers.Input(8, ))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1))

# 4) 컴파일
model.compile(loss='mse', optimizer='adam')

# 5) 구조 확인
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [29]:
# 3. 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   restore_best_weights=True)

model.fit(train_x, train_y, validation_split=0.15, callbacks=[es], epochs=500, verbose=1)

Epoch 1/500
384/384 [==============================] - 1s 3ms/step - loss: 1.0101 - val_loss: 0.6207
Epoch 2/500
384/384 [==============================] - 1s 3ms/step - loss: 0.5594 - val_loss: 0.5659
Epoch 3/500
384/384 [==============================] - 1s 3ms/step - loss: 0.5096 - val_loss: 0.5242
Epoch 4/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4761 - val_loss: 0.4980
Epoch 5/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4521 - val_loss: 0.4824
Epoch 6/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4400 - val_loss: 0.4708
Epoch 7/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4282 - val_loss: 0.4628
Epoch 8/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4229 - val_loss: 0.5148
Epoch 9/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4179 - val_loss: 0.4651
Epoch 10/500
384/384 [==============================] - 1s 3ms/step - loss: 0.4093 - val_lo